In [74]:
import pandas as pd
from pathlib import Path
import os
import html

Data pre-processing

In [75]:
ORIGINAL_DIR = Path('original')
CLEANED_DIR = Path('cleaned')

PO_GOO_DIR = Path('po-en-goo')
PO_DL_DIR = Path('po-en-dl')
FR_GOO_DIR = Path('fr-en-goo')
FR_DL_DIR = Path('fr-en-dl')
EN1_DIR = Path('en1')
EN2_DIR = Path('en2')

DIRS = [PO_GOO_DIR, PO_DL_DIR, FR_GOO_DIR, FR_DL_DIR, EN1_DIR, EN2_DIR]

In [76]:
# Find out the replacement by checking the vocabularies in parellel chapters for confirmation
TO_REPLACE = {
    'ö': 'o',
    'é': 'e',
    'ë': 'e',
    'ï': 'i',
    'ą': 'a',
    'í': 'i',
    'æ': 'ae',
    'ð': 'ny', # Original - fr-dl[8] 'caðon', changed to based on po-en-goo 'Canyon'
    'à': 'a',
    'å': 'a',
    'â': 'a',
    "'": '', # Replace apostrophe with nothing to protect contractions like don't -> dont
    '’': '',
    ' `': '',
}

In [77]:

allowed="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 \n"
def get_not_allowed(txt):
    others = set()
    for i in txt:
        if i not in allowed:
            others.add(i)
    return others

In [78]:
# Check special characters by checking each translated texts.
others = set()
for p in DIRS:
    for i in (ORIGINAL_DIR / p).iterdir(): # Checked by each directory contained target data
        with open(i) as f:
            txt = f.read()
            # Change the html punctuation format to the normal one
            txt = html.unescape(txt)
        others = others.union(get_not_allowed(txt))
print(others)

{"'", '.', '!', '»', '"', ':', 'â', ')', ',', '?', '³', 'ö', 'ë', '•', 'é', 'ï', ';', 'ą', '‘', 'í', '\x0c', '*', 'æ', '”', '^', 'å', '“', '(', 'ð', '`', '’', '–', 'à', '—', '…', '-', '_'}


In [79]:
for i in DIRS:
    for j in (ORIGINAL_DIR / i).iterdir():
        with open(j) as f:
            txt = f.read()
        
        # Change the html punctuation format to the normal one
        txt = html.unescape(txt)

        # Replace special characters
        for key in TO_REPLACE.keys():
            txt = txt.replace(key, TO_REPLACE[key])

        # Remove special characters
        for each in others:
            txt = txt.replace(each, ' ')
        
        if not (CLEANED_DIR / i).is_dir():
            (CLEANED_DIR / i).mkdir(parents=True)

        with open(CLEANED_DIR / i / j.name, 'w') as f:
            f.write(txt)
            